[View in Colaboratory](https://colab.research.google.com/github/BLUENCE/TIL/blob/master/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb)

# 추천시스템 

참고자료 : http://www.fun-coding.org/recommend_basic1.html 

In [0]:
# 기본적인 모듈 설치

import pandas as pd
import numpy as np

## 전체 추천 (popularity)

In [0]:
# 영화에 대한 평가 점수를 dictionary 형태로 입력 ( 일반적으로 영화평점은 딕셔너리 형태로 하는 경우가 많습니다. )

ratings={
     'A':{'달콤한인생':5,'범죄도시':3,'도둑들':3,'모아나':4},
     'B':{'달콤한인생':5,'범죄도시':1,'도둑들':4,'모아나':4},
     'C':{'달콤한인생':0,'범죄도시':4,'도둑들':5,'모아나':3},
     'D':{'달콤한인생':2,'범죄도시':1,'도둑들':5,'모아나':4}
}

In [3]:
# 데이터 확인

ratings['A']
# ratings['A']['달콤한인생']
# ratings['A'].keys()

{'달콤한인생': 5, '도둑들': 3, '모아나': 4, '범죄도시': 3}

In [5]:
ratings['A']['달콤한인생']

5

In [6]:
ratings['A'].keys()

dict_keys(['달콤한인생', '범죄도시', '도둑들', '모아나'])

**Work) 평점을 기준으로 두 영화를 추천해주세요! ** => 전체그룹 추천 

*   항목 추가
*   항목 추가




In [0]:
dat = pd.DataFrame(ratings)
dat

,A,B,C,D
달콤한인생,5,5,0,2
도둑들,3,4,5,5
모아나,4,4,3,4
범죄도시,3,1,4,1


In [0]:
tmp = dat.apply(np.mean, axis = 1)
tmp

달콤한인생    3.00
도둑들      4.25
모아나      3.75
범죄도시     2.25
dtype: float64

In [0]:
rate = tmp.sort_values(ascending = False)
rate

도둑들      4.25
모아나      3.75
달콤한인생    3.00
범죄도시     2.25
dtype: float64

In [0]:
# top 2개
rate[:2]

도둑들    4.25
모아나    3.75
dtype: float64

#### 참고 코드

In [0]:
# 영화별로 총 평점 구하기

movie_dict = dict()

for key_name in ratings:
    for movie_name in ratings[key_name].keys():
        if movie_name not in movie_dict:
            movie_dict[movie_name] = ratings[key_name][movie_name]
        else:
            movie_dict[movie_name] = (movie_dict[movie_name] + ratings[key_name][movie_name]) 
         

*  **for key_name in ratings : **  key_name 순으로 반복문 진행
  *  **  for movie_name in ratings[key_name].keys() :  **  영화순으로 반복문 진행
    * ** if movie_name not in movie_dict:  **'movie_dict' 안에 이 영화가 없다면 if문 진행
      * **movie_dict[movie_name] = ratings[key_name][movie_name] : **move_dict['영화제목'] 이 안에 영화 평점을 새롭게 넣어줘
    * **else:** 'movie_dict' 안에 영화가 있다면 else 문 진행
      * ** movie_dict[movie_name] = (movie_dict[movie_name] + ratings[key_name][movie_name])  :  ** move_dict['영화제목'] 이 안에 영화 평점을 계속 넣어줘

In [40]:
# 영화별로 평균 평점 구하기!

for movie_name in ratings['A'].keys():
    movie_dict[movie_name] = movie_dict[movie_name] / 4
    
movie_dict

{'달콤한인생': 3.0, '도둑들': 4.25, '모아나': 3.75, '범죄도시': 2.25}

In [41]:
# 점수가 높은 순으로 정렬하기

import operator

sorted_x = sorted(movie_dict.items(), key = operator.itemgetter(1), reverse=True)
sorted_x

[('도둑들', 4.25), ('모아나', 3.75), ('달콤한인생', 3.0), ('범죄도시', 2.25)]

dictionary는 리스트나 튜플이랑 다르게 순서가 없음! 따라서 dict를 튜플의 리스트로 표현 한 다음에 정렬해야 함!

* itemgetter : 딕셔너리 정렬 연산자 (1 : 두번째 값으로 정렬 하겠다 , 0: key 로 정렬하겠다)

In [0]:
# 평점이 가장 높은 두 영화를 출력하기
sorted_x[:2]

[('도둑들', 4.25), ('모아나', 3.75)]

## 협업필터링 (Collaborate Filtering)

해당 사용자와 유사한 사용자를 먼저 찾은 후 ! 사용자의 유사도를 바탕으로 가중치를 줘서 item점수 계산해서 추천 

In [0]:
# 영화에 대한 평가 점수가 다음과 같다

movie2={
     'A':{'달콤한인생':5,'범죄도시':3,'모아나':4},
     'B':{'달콤한인생':5,'범죄도시':1,'도둑들':4,'모아나':4, '서치':3},
     'C':{'달콤한인생':1,'범죄도시':4,'도둑들':5,'모아나':3, '명당':2},
     'D':{'달콤한인생':2,'범죄도시':1}
}

### User-based CF
** Work) User-based CF를 통해 D에게 적절한 영화를 추천해주세요 **

In [0]:
# Step1) User - Item matrix 만들기

In [0]:
df_movie2 = pd.DataFrame.from_dict(movie2)  #  orient='index' / orient를 index로 주면 유저-아이템 매트릭스형식으로 만들 수 있다

In [104]:
df_movie2

,A,B,C,D
달콤한인생,5.0,5.0,1.0,2.0
도둑들,NaN,4.0,5.0,NaN
명당,NaN,NaN,2.0,NaN
모아나,4.0,4.0,3.0,NaN
범죄도시,3.0,1.0,4.0,1.0
서치,NaN,3.0,NaN,NaN


In [0]:
df_movie2 = df_movie2.fillna(0).astype(int)  # Nan 값은 0으로 채워주고 그냥 int형으로 변환해준다. 


In [112]:
df_movie2['B']

달콤한인생    5
도둑들      4
명당       0
모아나      4
범죄도시     1
서치       3
Name: B, dtype: int64

In [0]:
# Step2) 'D' 유저와 다른 유저들의 유사도 측정 (상관계수, 유클리디안거리, 코사인유사도 등등이 존재함 

In [120]:
corr = df_movie2.corr()  # 각 컬럼간의 코사인 유사도 구하기 
corr

,A,B,C,D
A,1.000000,0.451908,-0.046881,0.733799
B,0.451908,1.000000,-0.082624,0.307923
C,-0.046881,-0.082624,1.000000,-0.191663
D,0.733799,0.307923,-0.191663,1.000000


In [0]:
sim = corr['D']
sim[3] = 0  # 마지막 값은 어차피 안쓸거니까 그냥 0으로 바꿈

In [126]:
sim

A    0.733799
B    0.307923
C   -0.191663
D    0.000000
Name: D, dtype: float64

In [0]:
# Step3) 유사도를 기준으로 item 평점 계산하기

In [123]:
df_movie2

,A,B,C,D
달콤한인생,5,5,1,2
도둑들,0,4,5,0
명당,0,0,2,0
모아나,4,4,3,0
범죄도시,3,1,4,1
서치,0,3,0,0


In [134]:
5*0.733799  # 한번 곱해보자

3.668995

In [128]:
mat = df_movie2*sim
mat  # user별 유사도가 곱해진 값 

,A,B,C,D
달콤한인생,3.668997,1.539616,-0.191663,0.0
도둑들,0.000000,1.231693,-0.958315,0.0
명당,0.000000,0.000000,-0.383326,0.0
모아나,2.935198,1.231693,-0.574989,0.0
범죄도시,2.201398,0.307923,-0.766652,0.0
서치,0.000000,0.923770,-0.000000,0.0


In [138]:
result = mat.mean(axis=1)
result # 각 영화별로 더해서 평균(mean)을 내면 아래와 같다. 

달콤한인생    1.254237
도둑들      0.068344
명당      -0.095831
모아나      0.897975
범죄도시     0.435667
서치       0.230942
dtype: float64

In [0]:
# Step4) 'D' 에게 적절한 영화 추천

In [141]:
result.sort_values(ascending=False)

달콤한인생    1.254237
모아나      0.897975
범죄도시     0.435667
서치       0.230942
도둑들      0.068344
명당      -0.095831
dtype: float64

사용자 D는 달콤한 도시, 범죄도시를 이미 봤으므로 '모아나' 추천!

#### 참고 코드

In [0]:
# Step1) User - Item matrix 만들기
movie_df = pd.DataFrame.from_dict(movie, orient = 'index')

In [0]:
# NA는 0으로 우선 처리
nan_idx = np.isnan(movie_df)
movie_df[nan_idx] = 0
movie_df

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
A,5,3,4.0,0.0,0.0,0.0
B,5,1,4.0,4.0,3.0,0.0
C,1,4,3.0,5.0,0.0,2.0
D,2,1,0.0,0.0,0.0,0.0


In [0]:
# Step2) 'D' 유저와 다른 유저들의 유사도 측정
#        유저들의 유사도를 상관계수로 구함!

# 유사도 초기화!
similarity = [0,0,0,0]

for i, name in enumerate(movie_df.index.values):
  if name == 'D':
    similarity[i] = 0
  else:
    similarity[i] = np.corrcoef(movie_df.loc[name].values, movie_df.loc['D'].values)[1,0]

In [0]:
# 유사도 확인
similarity

[0.7337993857053428, 0.30792318479945885, -0.191662969499982, 0]

In [0]:
# Step3) 유사도를 기준으로 item 평점 계산하기

item = []
for col_name in movie_df.columns.values:
  tmp  = sum(similarity * movie_df[col_name])
  item.append(tmp)
item

[5.016949883024026,
 1.7426694639155595,
 3.591901373519261,
 0.2733778916979255,
 0.9237695543983766,
 -0.383325938999964]

In [0]:
# item dataframe으로 만들어 주기!
item_df = pd.DataFrame(item)
item_df.index = user_cf.columns.values
item_df.columns = ['ratings']
item_df

,ratings
달콤한인생,5.016950
범죄도시,1.742669
모아나,3.591901
도둑들,0.273378
서치,0.923770
명당,-0.383326


In [0]:
# 예측한 평점순으로 정렬!
item_df.sort_values(by=['ratings'], ascending=False)

,ratings
달콤한인생,5.016950
모아나,3.591901
범죄도시,1.742669
서치,0.923770
도둑들,0.273378
명당,-0.383326


In [0]:
# Step4) 'D' 에게 적절한 영화 추천
movie_df

## D에서 '모아나' 영화 추천 !

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
A,5,3,4.0,0.0,0.0,0.0
B,5,1,4.0,4.0,3.0,0.0
C,1,4,3.0,5.0,0.0,2.0
D,2,1,0.0,0.0,0.0,0.0


### Item-based CF
** Work) Item-based CF를 통해 D에게 적절한 영화를 추천해주세요 **

In [0]:
# Step1) User - Item matrix 만들기

In [148]:
df_movie2.T  # 위에 있던거 그냥 그대로 가져오면서 트랜스포즈시킴

,달콤한인생,도둑들,명당,모아나,범죄도시,서치
A,5,0,0,4,3,0
B,5,4,0,4,1,3
C,1,5,2,3,4,0
D,2,0,0,0,1,0


In [0]:
# Step2) '달콤한인생', '범죄도시' 를 기준(사용자 'D'가 이미 구해한 아이템)으로 item별 유사도 측정

In [163]:
# 달콤한 인생과 도둑들, 달콤한 인생과 명당...
# 범죄도시와 달콤한인생, 범죄도시와 도둑들... 이런식으로 구해야 함 

df_movie2.T.corr()

,달콤한인생,도둑들,명당,모아나,범죄도시,서치
달콤한인생,1.000000,-0.261291,-0.727607,0.619268,-0.350329,0.565916
도둑들,-0.261291,1.000000,0.697097,0.418472,0.316862,0.443607
명당,-0.727607,0.697097,1.000000,0.088045,0.777778,-0.333333
모아나,0.619268,0.418472,0.088045,1.000000,0.381529,0.440225
범죄도시,-0.350329,0.316862,0.777778,0.381529,1.000000,-0.555556
서치,0.565916,0.443607,-0.333333,0.440225,-0.555556,1.000000


In [0]:
item_corr = df_movie2.T.corr() # 각 영화별 코사인 유사도 구하기 

In [170]:
item_corr['달콤한인생']  # 달콤한 인생과 각 영화아의 유사도 보기 

달콤한인생    1.000000
도둑들     -0.261291
명당      -0.727607
모아나      0.619268
범죄도시    -0.350329
서치       0.565916
Name: 달콤한인생, dtype: float64

In [0]:
# Step3) 유사도를 기준으로 item 평점 계산하기

In [156]:
item_corr = item_corr['달콤한인생']*2 + item_corr['범죄도시']*1  # D가 달콤한 인생과 범죄도시에 2점과 1점을 주었으므로 가중치를 주기위해 곱해줌 
item_corr

달콤한인생    1.649671
도둑들     -0.205720
명당      -0.677436
모아나      1.620065
범죄도시     0.299342
서치       0.576277
dtype: float64

In [157]:
item_corr.sort_values(ascending = False)

달콤한인생    1.649671
모아나      1.620065
서치       0.576277
범죄도시     0.299342
도둑들     -0.205720
명당      -0.677436
dtype: float64

In [0]:
# Step4) 'D' 에게 적절한 영화 추천

달콤한인생과 범죄도시를 빼면 '모아나' 추천

#### 참고 코드

In [0]:
# Step1) User - Item matrix 만들기
movie_df = pd.DataFrame.from_dict(movie, orient = 'index')

In [0]:
# NA는 0으로 우선 처리
nan_idx = np.isnan(movie_df)
movie_df[nan_idx] = 0
movie_df

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
A,5,3,4.0,0.0,0.0,0.0
B,5,1,4.0,4.0,3.0,0.0
C,1,4,3.0,5.0,0.0,2.0
D,2,1,0.0,0.0,0.0,0.0


In [0]:
# Step2) '달콤한인생', '범죄도시' 를 기준으로 item별 유사도 측정
#        아이템들의 유사도를 상관계수로 구함!

corr_mat = movie_df.corr()
corr_mat

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
달콤한인생,1.000000,-0.350329,0.619268,-0.261291,0.565916,-0.727607
범죄도시,-0.350329,1.000000,0.381529,0.316862,-0.555556,0.777778
모아나,0.619268,0.381529,1.000000,0.418472,0.440225,0.088045
도둑들,-0.261291,0.316862,0.418472,1.000000,0.443607,0.697097
서치,0.565916,-0.555556,0.440225,0.443607,1.000000,-0.333333
명당,-0.727607,0.777778,0.088045,0.697097,-0.333333,1.000000


In [0]:
# 유사도 확인
D_similarity = corr_mat[['달콤한인생','범죄도시']]

,달콤한인생,범죄도시
달콤한인생,1.000000,0.350329
범죄도시,0.350329,1.000000
모아나,0.619268,0.381529
도둑들,0.261291,0.316862
서치,0.565916,0.555556
명당,0.727607,0.777778


In [0]:
# Step3) 유사도를 기준으로 item 평점 계산하기
# D는 '달콤한인생'에 2점, 범죄도시에 1점을 줬음

res = D_similarity['달콤한인생'] * 2  + D_similarity['범죄도시'] * 1
res

달콤한인생    1.649671
범죄도시     0.299342
모아나      1.620065
도둑들     -0.205720
서치       0.576277
명당      -0.677436
dtype: float64

In [0]:
# Step4) 'D' 에게 적절한 영화 추천

# 예측한 평점순으로 정렬!
res.sort_values(ascending=False)


## D한테 '모아나' 영화 추천 !

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
A,5,3,4.0,0.0,0.0,0.0
B,5,1,4.0,4.0,3.0,0.0
C,1,4,3.0,5.0,0.0,2.0
D,2,1,0.0,0.0,0.0,0.0


### Matrix Factorization

#### Matrix operator (행렬 연산) 

In [0]:
# matrix

A = np.matrix('3 1 ; 8 2')
A

matrix([[3, 1],
        [8, 2]])

In [0]:
B = np.matrix('6 1; 7 9')
B

matrix([[6, 1],
        [7, 9]])

In [0]:
# 행렬의 합
A + B

# 행렬의 뺄셈
A - B

# 행렬의 곱
A * B

# 전치 행렬
A.T

# 역행렬
# linalg = linear algebra (선행대수)
np.linalg.inv(A)

##### SVD

In [0]:
# 영화에 대한 평가 점수가 다음과 같다

movie={
     'A':{'달콤한인생':5,'범죄도시':3,'모아나':4},
     'B':{'달콤한인생':5,'범죄도시':1,'도둑들':4,'모아나':4, '서치':3},
     'C':{'달콤한인생':1,'범죄도시':4,'도둑들':5,'모아나':3, '명당':2},
     'D':{'달콤한인생':2,'범죄도시':1}
}

In [0]:
movie_df = pd.DataFrame.from_dict(movie, orient = 'index')

In [0]:
# NA는 0으로 우선 처리
nan_idx = np.isnan(movie_df)
movie_df[nan_idx] = 0
movie_df

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
A,5,3,4.0,0.0,0.0,0.0
B,5,1,4.0,4.0,3.0,0.0
C,1,4,3.0,5.0,0.0,2.0
D,2,1,0.0,0.0,0.0,0.0


In [0]:
# SVD 
np.linalg.svd(movie_df)

(array([[-0.52604049, -0.53733777, -0.59393496, -0.28599089],
        [-0.65725107, -0.16979472,  0.73429   ,  0.0029976 ],
        [-0.52375574,  0.8020186 , -0.28353424,  0.04532569],
        [-0.13031642, -0.19800008, -0.16633638,  0.95715506]]),
 array([11.707668  ,  5.04075299,  3.61760229,  1.19761133]),
 array([[-0.5723468 , -0.38100772, -0.53848755, -0.44823469, -0.16841554,
         -0.08947226],
        [-0.62086836,  0.24366723, -0.08381172,  0.66079693, -0.10105319,
          0.31821381],
        [ 0.02365328, -0.64904544, -0.07993764,  0.42002648,  0.60893095,
         -0.15675258],
        [ 0.45479646,  0.23670682, -0.83165217,  0.19924567,  0.00750895,
          0.07569349],
        [-0.22116867,  0.44233734, -0.05529217, -0.35436215,  0.76737442,
          0.19475327],
        [ 0.17500498, -0.35000997,  0.04375125, -0.14401201, -0.04132397,
          0.9069206 ]]))

In [0]:
U, D, V = np.linalg.svd(movie_df)

**Work) U * D * V 의 값이 원래의 R과 같아지는지 확인해보시오. **

#### 참고코드

In [0]:
# D 는 singular values로 나타남. matrix로 바꿔주자!
D_mat = np.zeros((movie_df.shape[0], movie_df.shape[1]))
D_mat

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [0]:
D_mat[:movie_df.shape[0], :movie_df.shape[0]] = np.diag(D)
print(D_mat)

[[11.707668    0.          0.          0.          0.          0.        ]
 [ 0.          5.04075299  0.          0.          0.          0.        ]
 [ 0.          0.          3.61760229  0.          0.          0.        ]
 [ 0.          0.          0.          1.19761133  0.          0.        ]]


In [0]:
# SVD 확인
# 행렬의 곱셈은 다양한 방법으로 가능 


U @ D_mat @ V
# np.dot(np.dot(U, D_mat), V)
# np.matmul(np.matmul(U, D_mat), V.T)

array([[ 5.00000000e+00,  3.00000000e+00,  4.00000000e+00,
        -4.18449776e-16,  1.30427274e-15, -6.56078660e-16],
       [ 5.00000000e+00,  1.00000000e+00,  4.00000000e+00,
         4.00000000e+00,  3.00000000e+00,  7.61055109e-16],
       [ 1.00000000e+00,  4.00000000e+00,  3.00000000e+00,
         5.00000000e+00,  5.96566641e-16,  2.00000000e+00],
       [ 2.00000000e+00,  1.00000000e+00,  3.13646059e-16,
        -5.54929202e-16,  3.33170233e-16, -3.71883018e-16]])

In [0]:
R = U @ D_mat @ V
R.round()

array([[ 5.,  3.,  4., -0.,  0., -0.],
       [ 5.,  1.,  4.,  4.,  3.,  0.],
       [ 1.,  4.,  3.,  5.,  0.,  2.],
       [ 2.,  1.,  0., -0.,  0., -0.]])

In [0]:
movie_df

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
A,5,3,4.0,0.0,0.0,0.0
B,5,1,4.0,4.0,3.0,0.0
C,1,4,3.0,5.0,0.0,2.0
D,2,1,0.0,0.0,0.0,0.0


**분해한 것을 다시 곱하니 원래의 R과 똑같아 졌음 **

#### singular value중 일부만 선택해서 matrix를 재구성 하자

In [0]:
# singular values의 갯수를 결정해주자!
# 보통은 전체의 80~90%에 해당하는 값으로 결정

np.cumsum(D)/np.sum(D)

array([0.54293574, 0.7766975 , 0.94446153, 1.        ])

** 3개의 singular values를 써야 해! **

**Work) 3개의 singular values만 뽑아서 새로운 R 행렬을 만들어보고, 원래의 R 행렬과 얼마나 다른지 확인해보자 (10분)**

#### 참고코드

In [0]:
# U : 4 * 3 , D : 3 * 3,  V: 3*6



U2 = U[:, 0:3]
U2

array([[-0.52604049, -0.53733777, -0.59393496],
       [-0.65725107, -0.16979472,  0.73429   ],
       [-0.52375574,  0.8020186 , -0.28353424],
       [-0.13031642, -0.19800008, -0.16633638]])

In [0]:
D2 = D_mat[0:3, 0:3]
D2

array([[11.707668  ,  0.        ,  0.        ],
       [ 0.        ,  5.04075299,  0.        ],
       [ 0.        ,  0.        ,  3.61760229]])

In [0]:
V2 = V[0:3,]
V2

array([[-0.5723468 , -0.38100772, -0.53848755, -0.44823469, -0.16841554,
        -0.08947226],
       [-0.62086836,  0.24366723, -0.08381172,  0.66079693, -0.10105319,
         0.31821381],
       [ 0.02365328, -0.64904544, -0.07993764,  0.42002648,  0.60893095,
        -0.15675258]])

In [0]:
R2 = U2 @ D2 @ V2
R2.round(1)

array([[ 5.2,  3.1,  3.7,  0.1,  0. ,  0. ],
       [ 5. ,  1. ,  4. ,  4. ,  3. , -0. ],
       [ 1. ,  4. ,  3. ,  5. , -0. ,  2. ],
       [ 1.5,  0.7,  1. , -0.2, -0. , -0.1]])

In [0]:
movie_df

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
A,5,3,4.0,0.0,0.0,0.0
B,5,1,4.0,4.0,3.0,0.0
C,1,4,3.0,5.0,0.0,2.0
D,2,1,0.0,0.0,0.0,0.0


**원본 데이터와 값이 조금은 달라지긴 하지만, 크게 변하진 않아. 즉 차원 축소해도 정보의 손실이 적어!**

** 큰 matrix를 작은 matrix로 줄일 수 있음 **

#### Work) svd를 이용한 영화 추천 

In [0]:
# 영화에 대한 평가 점수가 다음과 같다

movie={
     'A':{'달콤한인생':5,'범죄도시':3,'모아나':4},
     'B':{'달콤한인생':5,'범죄도시':1,'도둑들':4,'모아나':4, '서치':3},
     'C':{'달콤한인생':1,'범죄도시':4,'도둑들':5,'모아나':3, '명당':2},
     'D':{'달콤한인생':2,'범죄도시':1}
}

In [0]:
# SVD를 이용해 영화 추천을 하시오. (User 기준으로 할지, Item 기준으로 할지 본인의 선택하기)

In [0]:
#1) User matrix 이나 Item matrix를 만드시오

In [0]:
#2) User matrix로 User-User 유사도 matrix를 만드시오 

In [0]:
#3) 위에서 만든 유사도 matrix를 기준으로 아이템을 추천하시오

#### 참고코드

In [0]:
# SVD를 이용해 영화 추천을 하시오. (User 기준!)

# 위에서 구한 U2, D2, V2를 이용 하자!

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
A,5,3,4.0,0.0,0.0,0.0
B,5,1,4.0,4.0,3.0,0.0
C,1,4,3.0,5.0,0.0,2.0
D,2,1,0.0,0.0,0.0,0.0


In [0]:
#1) User matrix 만들기

D_sqrt = np.sqrt(D2)
user = U2 @ D_sqrt
user

array([[-1.79992483, -1.20641041, -1.12966403],
       [-2.24888113, -0.38121669,  1.39661924],
       [-1.79210724,  1.8006618 , -0.539282  ],
       [-0.44589677, -0.44454228, -0.31637172]])

In [0]:
#2) User matrix로 User-User 유사도 matrix를 만드시오 

user_df = pd.DataFrame(user)
user_df.index = movie_df.index
user_df

,0,1,2
A,-1.799925,-1.206410,-1.129664
B,-2.248881,-0.381217,1.396619
C,-1.792107,1.800662,-0.539282
D,-0.445897,-0.444542,-0.316372


In [0]:
# user_df.corr() ,  이렇게 하면 열들간의 유사도가 계산됨

user_simil =user_df.T.corr()
user_simil

,A,B,C,D
A,1.000000,0.919270,0.695738,0.595189
B,0.919270,1.000000,0.356830,0.863453
C,0.695738,0.356830,1.000000,-0.163116
D,0.595189,0.863453,-0.163116,1.000000


In [0]:
#3) 위에서 만든 유사도 matrix를 기준으로 아이템을 추천하시오

# D와 D의 유사도는 0으로 만들기
us = user_simil['D']
us[3] = 0
us

A    0.595189
B    0.863453
C   -0.163116
D    0.000000
Name: D, dtype: float64

In [0]:
item = []
for col_name in movie_df.columns.values:
  tmp  = sum(us * movie_df[col_name])
  item.append(tmp)
item

[7.130094556573452,
 1.9965545632352417,
 5.345219502188873,
 2.638230483133708,
 2.590359469855774,
 -0.326232857336263]

In [0]:
# item dataframe으로 만들어 주기!
item_df = pd.DataFrame(item)
item_df.index = movie_df.columns.values
item_df.columns = ['ratings']
item_df

,ratings
달콤한인생,7.130095
범죄도시,1.996555
모아나,5.345220
도둑들,2.638230
서치,2.590359
명당,-0.326233


In [0]:
# 예측한 평점순으로 정렬!
item_df.sort_values(by=['ratings'], ascending=False)

,ratings
달콤한인생,7.130095
모아나,5.345220
도둑들,2.638230
서치,2.590359
범죄도시,1.996555
명당,-0.326233


In [0]:
# Step4) 'D' 에게 적절한 영화 추천
movie_df

## D에서 '모아나' 영화 추천 !

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
A,5,3,4.0,0.0,0.0,0.0
B,5,1,4.0,4.0,3.0,0.0
C,1,4,3.0,5.0,0.0,2.0
D,2,1,0.0,0.0,0.0,0.0
